In [2]:
import mido
import queue
import time
import threading
import mido.backends.rtmidi

In [5]:
# Function to read MIDI file and add messages to a priority queue
def read_midi_to_queue(midi_file_path, midi_queue):
    midi_file = mido.MidiFile(midi_file_path)
    absolute_time = 0  # to keep track of the absolute time in seconds

    for msg in midi_file:
        if not msg.is_meta:
            absolute_time += msg.time
            midi_queue.put(
                (absolute_time, msg)
            )  # Add message with its absolute time to the queue


# Function to send messages to a MIDI port at the correct times
def send_midi_from_queue(midi_queue, midi_out_port):
    start_time = time.time()

    while not midi_queue.empty():
        absolute_time, msg = midi_queue.get()
        sleep_time = absolute_time - (time.time() - start_time)

        if sleep_time > 0:
            time.sleep(sleep_time)

        midi_out_port.send(msg)

In [6]:
# Initialize a priority queue
midi_queue = queue.PriorityQueue()

# Replace 'your_midi_file.mid' with the path to your MIDI file
midi_file_path = "../data/datasets/test/play/20231220-080-01_0000-0005.mid"

# Read the MIDI file and add messages to the queue
read_midi_to_queue(midi_file_path, midi_queue)

# Initialize a MIDI output port
# Replace 'Your MIDI Output Port' with the name of your actual MIDI output port
with mido.open_output("Disklavier") as midi_out_port:
    send_midi_from_queue(midi_queue, midi_out_port)